# Calibrate aerosol forcing

Rather than take the individual forcers, we'll take the Smith et al. (2021) approach of curve fitting precursors.

In [1]:
from fair import FAIR
from fair.interface import fill, initialise
from fair.io import read_properties

import matplotlib.pyplot as pl
import pandas as pd
import pooch
import numpy as np
from scipy.optimize import curve_fit

/Users/c.l.b.alexander/Documents/FAIR/fair_virtual_env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
climate_response_df = pd.read_csv('../data/fair-calibrations/4xCO2_energy_balance_ebm3.csv')

In [3]:
climate_response_df

,model,run,gamma,C1,C2,C3,kappa1,kappa2,kappa3,epsilon,sigma_eta,sigma_xi,F_4xCO2
0,UKESM1-0-LL,r1i1p1f2,3.548407,2.9173,11.283175,73.248724,0.655766,2.597878,0.612934,1.133709,0.497439,0.439134,7.378788


In [4]:
aerosol_df = pd.read_csv('../data/smith2023/ukesm_ghan2013_aerosol_forcing.csv', index_col=0)

In [5]:
erfari = aerosol_df['ERFari']

In [6]:
erfaci = aerosol_df['ERFaci']

In [7]:
scenarios = ['ssp245']
configs = ['UKESM1-0-LL']
species = ['BC', 'OC', 'Sulfur', 'Aerosol-cloud interactions']
species, properties = read_properties(species=species)

In [8]:
rcmip_emissions_file = pooch.retrieve(
    url="doi:10.5281/zenodo.4589756/rcmip-emissions-annual-means-v5-1-0.csv",
    known_hash="md5:4044106f55ca65b094670e7577eaf9b3",
)
emis_df = pd.read_csv(rcmip_emissions_file)

In [9]:
bc = emis_df.loc[(emis_df['Scenario']=='historical')&(emis_df['Variable']=='Emissions|BC')&(emis_df['Region']=='World'),'1850':'2014'].values.squeeze()
oc = emis_df.loc[(emis_df['Scenario']=='historical')&(emis_df['Variable']=='Emissions|OC')&(emis_df['Region']=='World'),'1850':'2014'].values.squeeze()
so2 = emis_df.loc[(emis_df['Scenario']=='historical')&(emis_df['Variable']=='Emissions|Sulfur')&(emis_df['Region']=='World'),'1850':'2014'].values.squeeze()

In [10]:
def erfaci_rootfinder(x, shape_so2, shape_bc, shape_oc, beta):
    f = FAIR(temperature_prescribed=True)
    f.define_time(1850, 2015, 1)
    f.define_scenarios(scenarios)
    f.define_configs(configs)
    f.define_species(species, properties)
    f.allocate()
    fill(f.climate_configs['ocean_heat_capacity'], climate_response_df.loc[0, 'C1':'C3'])
    fill(f.climate_configs['ocean_heat_transfer'], climate_response_df.loc[0, 'kappa1':'kappa3'])
    fill(f.climate_configs['deep_ocean_efficacy'], climate_response_df.loc[0, 'epsilon'])
    fill(f.climate_configs['gamma_autocorrelation'], climate_response_df.loc[0, 'gamma'])
    initialise(f.forcing, 0)
    initialise(f.temperature, 0)
    f.fill_species_configs(filename='../data/species_configs_properties_vanilla.csv')
    f.species_configs['baseline_emissions'].loc[dict(specie='Sulfur')] = 4.5444637755469
    f.species_configs['baseline_emissions'].loc[dict(specie='BC')] = 2.57112447874271
    f.species_configs['baseline_emissions'].loc[dict(specie='OC')] = 18.2268247866406
    
    # fill emissions and concentrations
    f.emissions.loc[dict(specie='Sulfur')] = x[0][:,None,None]
    f.emissions.loc[dict(specie='BC')] = x[1][:,None,None]
    f.emissions.loc[dict(specie='OC')] = x[2][:,None,None]

    f.temperature[:] = 0

    # these things we want to vary
#    f.species_configs['erfari_radiative_efficiency'].loc[dict(specie='Sulfur')] = alpha_so2
#    f.species_configs['erfari_radiative_efficiency'].loc[dict(specie='BC')] = alpha_bc
#    f.species_configs['erfari_radiative_efficiency'].loc[dict(specie='OC')] = alpha_oc
    f.species_configs['aci_shape'].loc[dict(specie='Sulfur')] = shape_so2
    f.species_configs['aci_shape'].loc[dict(specie='BC')] = shape_bc
    f.species_configs['aci_shape'].loc[dict(specie='OC')] = shape_oc
    f.species_configs['aci_scale'].loc[dict(config='UKESM1-0-LL')] = beta
    
    f.run(progress=False)
    out = 0.5*(f.forcing.values[1:, 0, 0, 3] + f.forcing.values[:-1, 0, 0, 3])
    return out

In [11]:
solution = curve_fit(erfaci_rootfinder, [so2, bc, oc], erfaci, bounds=((0, 0, 0, -np.inf), (np.inf, np.inf, np.inf, np.inf)))

In [12]:
solution

(array([ 3.01888730e-02,  1.56370423e-29,  6.55048144e-16, -8.22336269e-01]),
 array([[ 0.00076965, -0.00535939,  0.00178455,  0.00173151],
        [-0.00535939,  0.05052942, -0.01403209, -0.00803404],
        [ 0.00178455, -0.01403209,  0.00446642,  0.0026748 ],
        [ 0.00173151, -0.00803404,  0.0026748 ,  0.01071309]]))

In [13]:
species = ['BC', 'OC', 'Sulfur', 'Aerosol-radiation interactions']
species, properties = read_properties(species=species)

def erfari_rootfinder(x, alpha_so2, alpha_bc, alpha_oc):
    f = FAIR(temperature_prescribed=True)
    f.define_time(1850, 2015, 1)
    f.define_scenarios(scenarios)
    f.define_configs(configs)
    f.define_species(species, properties)
    f.allocate()
    fill(f.climate_configs['ocean_heat_capacity'], climate_response_df.loc[0, 'C1':'C3'])
    fill(f.climate_configs['ocean_heat_transfer'], climate_response_df.loc[0, 'kappa1':'kappa3'])
    fill(f.climate_configs['deep_ocean_efficacy'], climate_response_df.loc[0, 'epsilon'])
    fill(f.climate_configs['gamma_autocorrelation'], climate_response_df.loc[0, 'gamma'])
    initialise(f.forcing, 0)
    initialise(f.temperature, 0)
    f.fill_species_configs(filename='../data/species_configs_properties_vanilla.csv')
    f.species_configs['baseline_emissions'].loc[dict(specie='Sulfur')] = 4.5444637755469
    f.species_configs['baseline_emissions'].loc[dict(specie='BC')] = 2.57112447874271
    f.species_configs['baseline_emissions'].loc[dict(specie='OC')] = 18.2268247866406
    
    # fill emissions and concentrations
    f.emissions.loc[dict(specie='Sulfur')] = x[0][:,None,None]
    f.emissions.loc[dict(specie='BC')] = x[1][:,None,None]
    f.emissions.loc[dict(specie='OC')] = x[2][:,None,None]

    f.temperature[:] = 0

    # these things we want to vary
    f.species_configs['erfari_radiative_efficiency'].loc[dict(specie='Sulfur')] = alpha_so2
    f.species_configs['erfari_radiative_efficiency'].loc[dict(specie='BC')] = alpha_bc
    f.species_configs['erfari_radiative_efficiency'].loc[dict(specie='OC')] = alpha_oc
    
    f.run(progress=False)
    out = 0.5*(f.forcing.values[1:, 0, 0, 3] + f.forcing.values[:-1, 0, 0, 3])
    return out

In [14]:
solution = curve_fit(erfari_rootfinder, [so2, bc, oc], erfari)

In [15]:
solution

(array([-0.00283793,  0.01757432, -0.0028512 ]),
 array([[ 4.48943823e-08, -2.14043912e-06,  4.98410270e-07],
        [-2.14043912e-06,  1.33037331e-04, -3.77464525e-05],
        [ 4.98410270e-07, -3.77464525e-05,  1.20890418e-05]]))